In [0]:
/*

show create table apachedataingestion.lte.widetable_4g_qci_gold;

CREATE TABLE apachedataingestion.lte.widetable_4g_qci_gold2 (
  Created_DTM TIMESTAMP,
  Busy_Hour INT,
  NE_ID STRING,
  SYSTEM_ID BIGINT,
  LOCATION STRING,
  qci STRING,
  D_DTM TIMESTAMP,
  Timezone STRING,
  Site_ID INT,
  MARKET_NAME STRING,
  SUB_MARKET_ID INT,
  SUB_MARKET_NAME STRING,
  BAND_NAME STRING,
  dl_user_throughput_num DOUBLE,
  dl_user_throughput_denom DOUBLE,
  dl_rlc_data_volume DOUBLE,
  ul_rlc_data_volume DOUBLE,
  ERAB_Setup_Failure_Rate_NUM DOUBLE,
  ERAB_Setup_Failure_Rate_DENOM DOUBLE,
  Data_Drop_Rate_NUM DOUBLE,
  Data_Drop_Rate_DENOM DOUBLE)
USING delta
PARTITIONED BY (D_DTM)
TBLPROPERTIES (
  'delta.checkpoint.writeStatsAsJson' = 'false',
  'delta.checkpoint.writeStatsAsStruct' = 'true',
  'delta.enableDeletionVectors' = 'true',
  'delta.feature.deletionVectors' = 'supported',
  'delta.minReaderVersion' = '3',
  'delta.minWriterVersion' = '7')



select * from apachedataingestion.lte.widetable_4g_qci_gold2


delete from apachedataingestion.lte.widetable_4g_qci_gold2

select * from apachedataingestion.lte.widetable_4g_qci_gold2 where D_DTM like '2024-08-25%' and Busy_Hour > 1

ALTER TABLE apachedataingestion.lte.widetable_4g_qci_gold2 drop COLUMN datetime_utc;

*/




delete from apachedataingestion.lte.widetable_4g_qci_gold2
where D_DTM >= date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd") and D_DTM < date_format(date_add(now(), ({{ Days }})+1),"yyyy-MM-dd") 
;




insert into apachedataingestion.lte.widetable_4g_qci_gold2
with bh_conf as (
  select SUB_MARKET_ID,REGION_SILVER_TB, Busy_Hr,bhcode from hive_metastore.default.vz_conf_market A
  inner join (
    select DAILY,REGION,Busy_Hr, sum(bhcode) bhcode from (       
        select DAILY,REGION,LTE_Busy_Hour 	Busy_Hr,1 bhcode from snap.enb_iris.4g_busy_hour_kpi_gold where Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")
        union all
        select DAILY,REGION,VoLTE_Busy_Hour Busy_Hr,2 bhcode from snap.enb_iris.4g_busy_hour_kpi_gold where Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")
    ) x group by DAILY,REGION,Busy_Hr
  ) B on A.REGION_SILVER_TB=B.REGION
)
SELECT date(D_DTM) CalendarDate,hour(D_DTM) Hour24,
convert_timezone('UTC','America/Chicago',current_timestamp())  Created_DTM,
decode(bhcode,null,0,bhcode) Busy_Hr,
/*X.NE_ID,*/
case when left(X.NE_ID,5)=='UADPF' and length(Y.SUB_MARKET_ID) = 3 then  replace(concat(left(X.NE_ID,9),right(X.NE_ID,3)),'UADPF','eNB') when left(X.NE_ID,5)=='UADPF' and length(Y.SUB_MARKET_ID) = 2 then replace(concat(left(X.NE_ID,8),right(X.NE_ID,3)),'UADPF','eNB')  else X.NE_ID  end NE_ID,  
X.SYSTEM_ID2 SYSTEM_ID,
X.LOCATION,X.qci,D_DTM,Timezone,replace(SITE_ID,'N/A',null) SITE_ID,
/*case when length(ne_ID)=5 then substring(ne_id, 1, 2)
     when length(ne_ID)=6 then substring(ne_id, 1, 3)
     when length(ne_ID)=9 then substring(ne_id, 5, 2)
     when length(ne_ID)=10 then substring(ne_id,5, 3)
     when length(ne_ID)=16 then substring(ne_id, 7, 2)
     when length(ne_ID)=17 then substring(ne_id, 7, 3) end SITE_ID,*/ 
Y.MARKET_NAME,
Y.SUB_MARKET_ID,
Y.SUB_MARKET_NAME,
Y.BAND_NAME,     
max(dl_user_throughput_num) dl_user_throughput_num,
max(dl_user_throughput_denom) dl_user_throughput_denom,
max(dl_rlc_data_volume_num) dl_rlc_data_volume,
max(ul_rlc_data_volume_num) ul_rlc_data_volume,
max(ERAB_Setup_Failure_Rate_NUM) ERAB_Setup_Failure_Rate_NUM,
max(ERAB_Setup_Failure_Rate_DENOM) ERAB_Setup_Failure_Rate_DENOM,
max(Data_Drop_Rate_NUM) Data_Drop_Rate_NUM,
max(Data_Drop_Rate_DENOM) Data_Drop_Rate_DENOM
from ( 	 
SELECT  cast(replace(replace(K.NE_ID,'eNB_',''),'UADPF_','') as int) SYSTEM_ID2,* from (
        SELECT 
          COALESCE(M0.NE_ID,T1.NE_ID,T2.NE_ID,T3.NE_ID,T4.NE_ID) NE_ID,          
          COALESCE(M0.LOCATION,T1.LOCATION,T2.LOCATION,T3.LOCATION,T4.LOCATION) LOCATION,
          COALESCE(M0.qci,T1.qci,T2.qci,T3.qci,T4.qci) qci,
          COALESCE(M0.D_DTM,T1.D_DTM,T2.D_DTM,T3.D_DTM,T4.D_DTM) D_DTM,
          COALESCE(M0.Timezone,T1.Timezone,T2.Timezone,T3.Timezone,T4.Timezone) Timezone,
          COALESCE(M0.SITE_ID,T1.SITE_ID,T2.SITE_ID,T3.SITE_ID,T4.SITE_ID) SITE_ID,
          `dl user throughput_num` dl_user_throughput_num,
          `dl user throughput_denom` dl_user_throughput_denom,          
          `dl rlc data volume num` dl_rlc_data_volume_num,          
          `ul rlc data volume num` ul_rlc_data_volume_num,     
          (EstabInitAttNbr + EstabAddAttNbr - ErabAddFailNbr_CpCcInteraction) -  (EstabInitSuccNbr + EstabAddSuccNbr ) as ERAB_Setup_Failure_Rate_NUM,
          (EstabInitAttNbr + EstabAddAttNbr -ErabAddFailNbr_CpCcInteraction)  ERAB_Setup_Failure_Rate_DENOM,
          RelAbnormalbyEnbQci + RelAbnormalbyMmeActQci as Data_Drop_Rate_NUM,
          RelNormalbyEnbQci + RelAbnormalbyEnbQci + RelNormalbyMmeQci + RelAbnormalbyMmeQci as Data_Drop_Rate_DENOM
FROM (      
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,split(LOCATION,'/')[1] qci,SITE_ID,Timezone,
        sum(IpThruThpVoDLByte_Kbytes_SUM *8/1000 ) as `DL User Throughput_NUM`,
        sum(IpThruThpDLTime_ms_SUM /1000) as `DL User Throughput_DENOM`,
        sum(AirRlcDLByte_Kbytes_SUM)  `DL RLC Data Volume NUM`,
        sum(AirRlcULByte_Kbytes_SUM)  `UL RLC Data Volume NUM`  
        -- select *      
      from apachedataingestion.lte.airrlcpacket_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")    	
      and  split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI65','QCI66','QCI67','QCI69','QCI70','QCI132','QCI133')	 
      GROUP BY Daily,Hourly,NE_ID,LOCATION,SITE_ID,Timezone   
    ) M0			      
    full join  (    
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,split(LOCATION,'/')[1] qci,SITE_ID,Timezone,
        sum(RelAbnormalbyEnbQci_count_SUM) RelAbnormalbyEnbQci,
        sum(RelNormalbyEnbQci_count_SUM) RelNormalbyEnbQci
        -- select *
      from apachedataingestion.lte.erabreleasebyenbpercellperqci_lte_hr_silver  
      WHERE Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")    
      and  split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI65','QCI66','QCI67','QCI69','QCI70','QCI132','QCI133')	    
      GROUP BY Daily,Hourly,NE_ID,LOCATION,SITE_ID,Timezone
    ) T1 on M0.D_DTM=T1.D_DTM and M0.NE_ID=T1.NE_ID  and M0.LOCATION=T1.LOCATION and M0.qci=T1.qci
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,split(LOCATION,'/')[1] qci,SITE_ID,Timezone,
        sum(RelAbnormalbyMmeActQci_count_SUM) RelAbnormalbyMmeActQci,
        sum(RelNormalbyMmeQci_count_SUM) RelNormalbyMmeQci,
        sum(RelAbnormalbyMmeQci_count_SUM) RelAbnormalbyMmeQci 
        -- select *
      from apachedataingestion.lte.erabreleasebymmepercellperqci_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")     
      and  split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI65','QCI66','QCI67','QCI69','QCI70','QCI132','QCI133')	 
      GROUP BY Daily,Hourly,NE_ID,LOCATION,SITE_ID,Timezone     
    ) T2 on M0.D_DTM=T2.D_DTM and M0.NE_ID=T2.NE_ID  and M0.LOCATION=T2.LOCATION and M0.qci=T2.qci  
    full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION,split(LOCATION,'/')[1] qci,SITE_ID,Timezone,
        sum(EstabAddAttNbr_count_SUM) EstabAddAttNbr,
        sum(EstabAddSuccNbr_count_SUM) EstabAddSuccNbr,
        sum(ErabAddFailNbr_CpCcInteraction_count_SUM) ErabAddFailNbr_CpCcInteraction
      -- select *
      from apachedataingestion.lte.erabsetupadd_lte_hr_silver
      WHERE Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")    	      
      and  split(LOCATION,'/')[1] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI65','QCI66','QCI67','QCI69','QCI70','QCI132','QCI133')	 
      GROUP BY Daily,Hourly,NE_ID,LOCATION,SITE_ID,Timezone  
    ) T3 on M0.D_DTM=T3.D_DTM and M0.NE_ID=T3.NE_ID  and M0.LOCATION=T3.LOCATION and M0.qci=T3.qci
full join  (
      SELECT timestamp(concat(Daily,' ',Hourly,':00:00')) D_DTM,NE_ID,split(LOCATION,'/')[0] LOCATION, split(LOCATION,'/')[2] qci,SITE_ID,Timezone,
        sum(EstabInitAttNbr_count_SUM) EstabInitAttNbr,
        sum(EstabInitSuccNbr_count_SUM) EstabInitSuccNbr
        -- select *
      from apachedataingestion.lte.erabsetup_lte_hr_silver			        			
      WHERE Daily =  date_format(date_add(now(), {{ Days }}),"yyyy-MM-dd")         
      and  split(LOCATION,'/')[2] in ('QCI1','QCI5','QCI6','QCI7','QCI8','QCI9','QCI65','QCI66','QCI67','QCI69','QCI70','QCI132','QCI133')	 
      GROUP BY Daily,Hourly,NE_ID,split(LOCATION,'/')[0], split(LOCATION,'/')[2],SITE_ID,Timezone  
   ) T4 on M0.D_DTM=T4.D_DTM and M0.NE_ID=T4.NE_ID  and M0.LOCATION=T4.LOCATION and M0.qci=T4.qci     
  ) K            
) X
left join bh_conf B on cast(decode(len(SYSTEM_ID2),5,left(SYSTEM_ID2,2),10,left(SYSTEM_ID2,2),mod(left(SYSTEM_ID2,3),300) ) as int)=B.SUB_MARKET_ID and hour(D_DTM)=B.Busy_Hr
left join (
    select replace(ENODEB_ID,'VZ_','eNB_') NE_ID,concat('cNum',ECELL_ID) LOCATION,
    max(MARKET_NAME) MARKET_NAME,max(SUB_MARKET_ID) SUB_MARKET_ID,max(SUB_MARKET_NAME) SUB_MARKET_NAME,max(BAND_NAME) BAND_NAME 
    from hive_metastore.default.vz_conf_ecell_silver group by ENODEB_ID,ECELL_ID
) Y on X.NE_ID=Y.NE_ID and X.LOCATION=Y.LOCATION
Group by X.NE_ID,X.SYSTEM_ID2,X.LOCATION,X.qci,D_DTM,Timezone,SITE_ID,bhcode,Y.MARKET_NAME,Y.SUB_MARKET_ID,Y.SUB_MARKET_NAME,Y.BAND_NAME;






